# Rename existing files on lustre

In [1]:
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

import pandas as pd
from tqdm import tqdm

tenx = pd.read_csv("data/datasets_10x.csv", sep=";")

In [2]:
uid_mapping = dict(zip(tenx["uid_old"], tenx["uid"], strict=False))
uid_mapping

{'ooaie': '10u63',
 'nqe4e': '106xa',
 '56r5s': '10n1k',
 '1lhfw': '10ndm',
 'fulvo': '10nsr',
 'hp7qe': '10040',
 'oftpw': '10pui',
 'xxjnz': '108x2',
 'wxjgz': '10vek',
 'jqmx8': '10s7d',
 '6dl1f': '107z9',
 'a680h': '10sja',
 'aylse': '10i70',
 'twgn1': '10arp',
 'i3vq9': '1020d',
 'emmt4': '10k3x',
 '0bbcf': '10ae2',
 'ic0bo': '1074f',
 'x3fho': '108dr',
 'gsyej': '10wmu',
 'rqu2u': '10uxy',
 '6uvee': '10zwi',
 'f61rr': '10fyx',
 '4b6qm': '10tei',
 '31nta': '10jfe',
 'zr2gu': '10e5i',
 'y5py1': '1065r',
 '7unc5': '10caf',
 'sjmmx': '105k3',
 'a0xyt': '10ps4',
 'up5uh': '10ha9',
 'mfrkl': '102cy',
 'fh4bl': '10tsy',
 'w1tgu': '101t3',
 'qilz7': '10ah4',
 'wxc2q': '10zm9',
 'w7zg8': '10jr4',
 '4kt3d': '10pa3',
 '7z1k9': '107sr',
 '2549m': '103mf',
 'n0qwg': '107yk',
 'uhk1j': '1063i',
 '4slyf': '10u89',
 '8x7n0': '10hhx',
 'cdmnx': '10kyp',
 '9z0ng': '10fpp',
 'g9gmu': '106ws',
 'qs3a7': '10xa2',
 'oeuwa': '108d3',
 'qdtdp': '106s5',
 '2lfvx': '103uy',
 'tggfj': '10mda',
 'l679p': '1

In [ ]:
def gather_files(target_dir, extensions=None):
    """
    Gather all file paths in the target directory (recursively) that match the specified extensions.

    Parameters
    ----------
        target_dir (str): Path to the target directory.
        extensions (list, optional): List of file extensions to include (e.g., ['.txt', '.md']).
                                     If None, all files are included.

    Returns
    -------
        list: List of file paths to process.
    """
    files_to_process = []
    for root, _dirs, files in os.walk(target_dir):
        for name in files:
            if extensions:
                if not any(name.lower().endswith(ext) for ext in extensions):
                    continue
            file_path = os.path.join(root, name)
            files_to_process.append(file_path)
    return files_to_process


def gather_directories(target_dir):
    """
    Gather all directory paths in the target directory (recursively).

    Parameters
    ----------
        target_dir (str): Path to the target directory.

    Returns
    -------
        list: List of directory paths to process.
    """
    directories_to_process = []
    for root, dirs, _files in os.walk(target_dir):
        for name in dirs:
            dir_path = os.path.join(root, name)
            directories_to_process.append(dir_path)
    return directories_to_process


def rename_file(file_path, mapping):
    """
    Rename a single file by replacing occurrences of uid_old with uid in the filename.

    Parameters
    ----------
        file_path (str): Path to the file to be renamed.
        mapping (dict): Dictionary mapping 'uid_old' to 'uid'.

    Returns
    -------
        bool: True if renamed successfully, False otherwise.
    """
    directory, filename = os.path.split(file_path)
    new_filename = filename
    for uid_old, uid in mapping.items():
        new_filename = new_filename.replace(uid_old, uid)

    if new_filename != filename:
        new_file_path = os.path.join(directory, new_filename)
        try:
            if not os.path.exists(new_file_path):
                os.rename(file_path, new_file_path)
                print(f"Renamed file: {file_path} -> {new_file_path}")
                return True
            else:
                print(f"Skipping rename for {file_path}: {new_file_path} already exists.")
                return False
        except Exception as e:  # noqa: BLE001
            print(f"Error renaming file {file_path} to {new_file_path}: {e}")
            return False
    return False  # No renaming needed


def rename_directory(dir_path, mapping):
    """
    Rename a single directory by replacing occurrences of uid_old with uid in the directory name.

    Parameters
    ----------
        dir_path (str): Path to the directory to be renamed.
        mapping (dict): Dictionary mapping 'uid_old' to 'uid'.

    Returns
    -------
        bool: True if renamed successfully, False otherwise.
    """
    parent_dir, dir_name = os.path.split(dir_path)
    new_dir_name = dir_name
    for uid_old, uid in mapping.items():
        new_dir_name = new_dir_name.replace(uid_old, uid)

    if new_dir_name != dir_name:
        new_dir_path = os.path.join(parent_dir, new_dir_name)
        try:
            if not os.path.exists(new_dir_path):
                os.rename(dir_path, new_dir_path)
                print(f"Renamed directory: {dir_path} -> {new_dir_path}")
                return True
            else:
                print(f"Skipping rename for {dir_path}: {new_dir_path} already exists.")
                return False
        except Exception as e:  # noqa: BLE001
            print(f"Error renaming directory {dir_path} to {new_dir_path}: {e}")
            return False
    return False  # No renaming needed


def rename_files_parallel(files, mapping):
    """
    Rename files in parallel by replacing uid_old with uid in filenames.

    Parameters
    ----------
        files (list): List of file paths to rename.
        mapping (dict): Dictionary mapping 'uid_old' to 'uid'.

    Returns
    -------
        int: Number of files successfully renamed.
    """
    renamed_count = 0
    with ProcessPoolExecutor() as executor:
        # Submit all rename tasks
        future_to_file = {executor.submit(rename_file, file, mapping): file for file in files}

        for future in tqdm(as_completed(future_to_file), total=len(future_to_file), desc="Renaming files"):
            file = future_to_file[future]
            try:
                result = future.result()
                if result:
                    renamed_count += 1
            except Exception as e:  # noqa: BLE001
                print(f"Error processing file {file}: {e}")
    return renamed_count


def rename_directories_parallel(directories, mapping):
    """
    Rename directories in parallel by replacing uid_old with uid in directory names.

    Parameters
    ----------
        directories (list): List of directory paths to rename.
        mapping (dict): Dictionary mapping 'uid_old' to 'uid'.

    Returns
    -------
        int: Number of directories successfully renamed.
    """
    renamed_count = 0
    with ProcessPoolExecutor() as executor:
        # Submit all rename tasks
        future_to_dir = {executor.submit(rename_directory, dir_path, mapping): dir_path for dir_path in directories}

        for future in tqdm(as_completed(future_to_dir), total=len(future_to_dir), desc="Renaming directories"):
            dir_path = future_to_dir[future]
            try:
                result = future.result()
                if result:
                    renamed_count += 1
            except Exception as e:  # noqa: BLE001
                print(f"Error processing directory {dir_path}: {e}")
    return renamed_count

In [5]:
target_directory = "/lustre/groups/ml01/projects/2024_spatialdata_db/data"

In [6]:
extensions = None  # Set to None to process all files

files_to_process = gather_files(target_directory)
directories_to_process = gather_directories(target_directory)

In [8]:
# Sort directories from deepest to shallowest to avoid path issues during renaming
directories_to_process_sorted = sorted(directories_to_process, key=lambda x: x.count(os.sep), reverse=True)

In [17]:
renamed_dirs_count = rename_directories_parallel(directories_to_process_sorted, uid_mapping)

Renaming directories:  95%|█████████▍| 24690/25999 [00:05<00:00, 4990.71it/s]

Error renaming directory /lustre/groups/ml01/projects/2024_spatialdata_db/data/ibiof__10X__Visium__Human__ovary__20201027__v1.2.0/ibiof__10X__Visium__Human__ovary__20201027__v1.2.0.zarr to /lustre/groups/ml01/projects/2024_spatialdata_db/data/ibiof__10X__Visium__Human__ovary__20201027__v1.2.0/10j0z__10X__Visium__Human__ovary__20201027__v1.2.0.zarr: [Errno 2] No such file or directory: '/lustre/groups/ml01/projects/2024_spatialdata_db/data/ibiof__10X__Visium__Human__ovary__20201027__v1.2.0/ibiof__10X__Visium__Human__ovary__20201027__v1.2.0.zarr' -> '/lustre/groups/ml01/projects/2024_spatialdata_db/data/ibiof__10X__Visium__Human__ovary__20201027__v1.2.0/10j0z__10X__Visium__Human__ovary__20201027__v1.2.0.zarr'Error renaming directory /lustre/groups/ml01/projects/2024_spatialdata_db/data/zr2gu__10X__Visium__Human__colon__20231005__v2.1.0__ControlReplicate1/zr2gu__10X__Visium__Human__colon__20231005__v2.1.0__ControlReplicate1.zarr to /lustre/groups/ml01/projects/2024_spatialdata_db/data/zr2

Renaming directories:  97%|█████████▋| 25193/25999 [00:05<00:00, 4436.28it/s]




Error renaming directory /lustre/groups/ml01/projects/2024_spatialdata_db/data/twgn1__10X__Xenium__Human__tonsil__20240305__v1.9.0__Reactivefollicularhyperplasia/twgn1__10X__Xenium__Human__tonsil__20240305__v1.9.0__Reactivefollicularhyperplasia.zarr to /lustre/groups/ml01/projects/2024_spatialdata_db/data/twgn1__10X__Xenium__Human__tonsil__20240305__v1.9.0__Reactivefollicularhyperplasia/10arp__10X__Xenium__Human__tonsil__20240305__v1.9.0__Reactivefollicularhyperplasia.zarr: [Errno 2] No such file or directory: '/lustre/groups/ml01/projects/2024_spatialdata_db/data/twgn1__10X__Xenium__Human__tonsil__20240305__v1.9.0__Reactivefollicularhyperplasia/twgn1__10X__Xenium__Human__tonsil__20240305__v1.9.0__Reactivefollicularhyperplasia.zarr' -> '/lustre/groups/ml01/projects/2024_spatialdata_db/data/twgn1__10X__Xenium__Human__tonsil__20240305__v1.9.0__Reactivefollicularhyperplasia/10arp__10X__Xenium__Human__tonsil__20240305__v1.9.0__Reactivefollicularhyperplasia.zarr'Error renaming directory 

Renaming directories:  99%|█████████▉| 25699/25999 [00:05<00:00, 4590.94it/s]

Error renaming directory /lustre/groups/ml01/projects/2024_spatialdata_db/data/oftpw__10X__Xenium__Human__bone_bone_marrow__20240403__v1.9.0__Non-diseasedFemurBone/oftpw__10X__Xenium__Human__bone_bone_marrow__20240403__v1.9.0__Non-diseasedFemurBone.zarr to /lustre/groups/ml01/projects/2024_spatialdata_db/data/oftpw__10X__Xenium__Human__bone_bone_marrow__20240403__v1.9.0__Non-diseasedFemurBone/10pui__10X__Xenium__Human__bone_bone_marrow__20240403__v1.9.0__Non-diseasedFemurBone.zarr: [Errno 2] No such file or directory: '/lustre/groups/ml01/projects/2024_spatialdata_db/data/oftpw__10X__Xenium__Human__bone_bone_marrow__20240403__v1.9.0__Non-diseasedFemurBone/oftpw__10X__Xenium__Human__bone_bone_marrow__20240403__v1.9.0__Non-diseasedFemurBone.zarr' -> '/lustre/groups/ml01/projects/2024_spatialdata_db/data/oftpw__10X__Xenium__Human__bone_bone_marrow__20240403__v1.9.0__Non-diseasedFemurBone/10pui__10X__Xenium__Human__bone_bone_marrow__20240403__v1.9.0__Non-diseasedFemurBone.zarr'
Error renam

Renaming directories: 100%|██████████| 25999/25999 [00:05<00:00, 4572.48it/s]

Error renaming directory /lustre/groups/ml01/projects/2024_spatialdata_db/data/u8rpv__10X__Visium__Mouse__brain__20200623__v1.1.0/u8rpv__10X__Visium__Mouse__brain__20200623__v1.1.0.zarr to /lustre/groups/ml01/projects/2024_spatialdata_db/data/u8rpv__10X__Visium__Mouse__brain__20200623__v1.1.0/100wk__10X__Visium__Mouse__brain__20200623__v1.1.0.zarr: [Errno 2] No such file or directory: '/lustre/groups/ml01/projects/2024_spatialdata_db/data/u8rpv__10X__Visium__Mouse__brain__20200623__v1.1.0/u8rpv__10X__Visium__Mouse__brain__20200623__v1.1.0.zarr' -> '/lustre/groups/ml01/projects/2024_spatialdata_db/data/u8rpv__10X__Visium__Mouse__brain__20200623__v1.1.0/100wk__10X__Visium__Mouse__brain__20200623__v1.1.0.zarr'Error renaming directory /lustre/groups/ml01/projects/2024_spatialdata_db/data/l7fdr__10X__Visium__Mouse__brain__20220329__v1.3.0/l7fdr__10X__Visium__Mouse__brain__20220329__v1.3.0.zarr to /lustre/groups/ml01/projects/2024_spatialdata_db/data/l7fdr__10X__Visium__Mouse__brain__2022032

Skipping rename for /lustre/groups/ml01/projects/2024_spatialdata_db/data/f0fww__10X__Visium__Mouse__lung_brain__20230221__v2.0.1__BrainReplicate4: /lustre/groups/ml01/projects/2024_spatialdata_db/data/10sph__10X__Visium__Mouse__lung_brain__20230221__v2.0.1__BrainReplicate4 already exists.
Skipping rename for /lustre/groups/ml01/projects/2024_spatialdata_db/data/cdmnx__10X__Xenium__Mouse__brain__20230713__v1.4.0__TgCRND82.5months: /lustre/groups/ml01/projects/2024_spatialdata_db/data/10kyp__10X__Xenium__Mouse__brain__20230713__v1.4.0__TgCRND82.5months already exists.Skipping rename for /lustre/groups/ml01/projects/2024_spatialdata_db/data/u8rpv__10X__Visium__Mouse__brain__20200623__v1.1.0: /lustre/groups/ml01/projects/2024_spatialdata_db/data/100wk__10X__Visium__Mouse__brain__20200623__v1.1.0 already exists.Skipping rename for /lustre/groups/ml01/projects/2024_spatialdata_db/data/h5tfz__10X__Visium__Human__breast__20230106__v2.0.1: /lustre/groups/ml01/projects/2024_spatialdata_db/data/

In [18]:
renamed_files_count = rename_files_parallel(files_to_process, uid_mapping)

Renaming files:   2%|▏         | 3320/141703 [00:01<00:29, 4651.77it/s]

Error renaming file /lustre/groups/ml01/projects/2024_spatialdata_db/data/vbdbc__10X__Visium__Mouse__kidney__20191202__v1.0.0/vbdbc__10X__Visium__Mouse__kidney__20191202__v1.0.0.contents to /lustre/groups/ml01/projects/2024_spatialdata_db/data/vbdbc__10X__Visium__Mouse__kidney__20191202__v1.0.0/1031a__10X__Visium__Mouse__kidney__20191202__v1.0.0.contents: [Errno 2] No such file or directory: '/lustre/groups/ml01/projects/2024_spatialdata_db/data/vbdbc__10X__Visium__Mouse__kidney__20191202__v1.0.0/vbdbc__10X__Visium__Mouse__kidney__20191202__v1.0.0.contents' -> '/lustre/groups/ml01/projects/2024_spatialdata_db/data/vbdbc__10X__Visium__Mouse__kidney__20191202__v1.0.0/1031a__10X__Visium__Mouse__kidney__20191202__v1.0.0.contents'


Renaming files:   3%|▎         | 3796/141703 [00:01<00:29, 4635.31it/s]

Error renaming file /lustre/groups/ml01/projects/2024_spatialdata_db/data/cyxpa__10X__Visium__Human__breast__20210609__v1.3.0/cyxpa__10X__Visium__Human__breast__20210609__v1.3.0.contents to /lustre/groups/ml01/projects/2024_spatialdata_db/data/cyxpa__10X__Visium__Human__breast__20210609__v1.3.0/10jcz__10X__Visium__Human__breast__20210609__v1.3.0.contents: [Errno 2] No such file or directory: '/lustre/groups/ml01/projects/2024_spatialdata_db/data/cyxpa__10X__Visium__Human__breast__20210609__v1.3.0/cyxpa__10X__Visium__Human__breast__20210609__v1.3.0.contents' -> '/lustre/groups/ml01/projects/2024_spatialdata_db/data/cyxpa__10X__Visium__Human__breast__20210609__v1.3.0/10jcz__10X__Visium__Human__breast__20210609__v1.3.0.contents'
Error renaming file /lustre/groups/ml01/projects/2024_spatialdata_db/data/09ohz__10X__Visium__Mouse__brain__20191202__v1.0.0/09ohz__10X__Visium__Mouse__brain__20191202__v1.0.0.contents to /lustre/groups/ml01/projects/2024_spatialdata_db/data/09ohz__10X__Visium__Mo

Renaming files:   3%|▎         | 4410/141703 [00:01<00:27, 5077.43it/s]

...

Renaming files:  99%|█████████▉| 140219/141703 [00:30<00:00, 5252.17it/s]

Error renaming file /lustre/groups/ml01/projects/2024_spatialdata_db/data/mt3dy__10X__Visium__Mouse__brain__20200623__v1.1.0/mt3dy__10X__Visium__Mouse__brain__20200623__v1.1.0.contents to /lustre/groups/ml01/projects/2024_spatialdata_db/data/mt3dy__10X__Visium__Mouse__brain__20200623__v1.1.0/10wfn__10X__Visium__Mouse__brain__20200623__v1.1.0.contents: [Errno 2] No such file or directory: '/lustre/groups/ml01/projects/2024_spatialdata_db/data/mt3dy__10X__Visium__Mouse__brain__20200623__v1.1.0/mt3dy__10X__Visium__Mouse__brain__20200623__v1.1.0.contents' -> '/lustre/groups/ml01/projects/2024_spatialdata_db/data/mt3dy__10X__Visium__Mouse__brain__20200623__v1.1.0/10wfn__10X__Visium__Mouse__brain__20200623__v1.1.0.contents'


Renaming files:  99%|█████████▉| 140754/141703 [00:30<00:00, 4900.64it/s]

Error renaming file /lustre/groups/ml01/projects/2024_spatialdata_db/data/zqxk7__10X__Visium__Human__large_intestine__20220328__v1.3.0/zqxk7__10X__Visium__Human__large_intestine__20220328__v1.3.0.contents to /lustre/groups/ml01/projects/2024_spatialdata_db/data/zqxk7__10X__Visium__Human__large_intestine__20220328__v1.3.0/1000j__10X__Visium__Human__large_intestine__20220328__v1.3.0.contents: [Errno 2] No such file or directory: '/lustre/groups/ml01/projects/2024_spatialdata_db/data/zqxk7__10X__Visium__Human__large_intestine__20220328__v1.3.0/zqxk7__10X__Visium__Human__large_intestine__20220328__v1.3.0.contents' -> '/lustre/groups/ml01/projects/2024_spatialdata_db/data/zqxk7__10X__Visium__Human__large_intestine__20220328__v1.3.0/1000j__10X__Visium__Human__large_intestine__20220328__v1.3.0.contents'


Renaming files: 100%|██████████| 141703/141703 [00:30<00:00, 4621.33it/s]
